<a href="https://colab.research.google.com/github/petermerkwaardig/studiobierkade/blob/main/Hoeveelkrijgjemee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Ontslagvergoedingcalculator**

1.   Druk op de play-knop om de berekening te starten
2.   Vul start- en einddatum in van je dienstverband.
3.   Geef het bruto jaarinkomen op voor elk jaar van je dienstverband.

Klik op 'Bereken' en je ziet je bruto maandsalaris bij Lifetri en je ontslagvergoeding (volgens het sociaal plan)


In [ ]:
# @title Hoeveel krijg je mee? { run: "auto", vertical-output: true, display-mode: "form" }
# @title
# Importeer de benodigde bibliotheken
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import datetime

# Functies voor berekeningen
def calculate_average_monthly_salary(yearly_salaries):
    total_salary = sum(yearly_salaries)
    average_monthly_salary = total_salary / (len(yearly_salaries) * 12)
    return round(average_monthly_salary, 2)

def calculate_severance_payment(start_date, end_date, monthly_salary):
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    end_date = datetime.strptime(end_date, '%Y-%m-%d')
    total_months = (end_date.year - start_date.year) * 12 + end_date.month - start_date.month
    years = total_months // 12
    months = total_months % 12
    partial_year = months / 12
    transitievergoeding_per_year = (1/3) * monthly_salary
    transitievergoeding = (years + partial_year) * transitievergoeding_per_year
    correctiefactor = 1.8
    severance_payment = transitievergoeding * correctiefactor
    return round(severance_payment, 2)

# Callback functie om jaarinkomen widgets te genereren
def generate_yearly_income_widgets(change):
    # Wis de bestaande output
    clear_output(wait=True)
    display(start_date_widget, end_date_widget, button_generate_income_widgets)

    # Haal de start- en einddatums op
    start_date = start_date_widget.value
    end_date = end_date_widget.value

    if not start_date or not end_date:
        print("Voer zowel een startdatum als einddatum in.")
        return

    try:
        start_date = datetime.strptime(start_date, '%Y-%m-%d')
        end_date = datetime.strptime(end_date, '%Y-%m-%d')
    except ValueError:
        print("Gebruik het juiste datumformaat (YYYY-MM-DD).")
        return

    if start_date > end_date:
        print("Startdatum kan niet later zijn dan einddatum.")
        return

    # Bereken de jaren van dienst
    start_year = start_date.year
    end_year = end_date.year

    # Genereer widgets voor de jaren dat de werknemer in dienst was
    global salary_widgets
    salary_widgets = []
    for year in range(start_year, end_year + 1):
        salary_widget = widgets.FloatText(value=0, description=f'Bruto {year}:')
        salary_widgets.append(salary_widget)
        display(salary_widget)

    # Weergeven van de bereken knop
    display(button_calculate)

# Functie om berekeningen uit te voeren
def on_calculate_button_click(b):
    try:
        yearly_salaries = [widget.value for widget in salary_widgets]
        average_monthly_salary = calculate_average_monthly_salary(yearly_salaries)
        severance_payment = calculate_severance_payment(start_date_widget.value, end_date_widget.value, average_monthly_salary)
        print(f"Gemiddeld bruto maandsalaris: €{average_monthly_salary}")
        print(f"Ontslagvergoeding: €{severance_payment}")
    except Exception as e:
        print(f"Fout bij berekenen: {e}")

# Widgets voor datuminvoer
start_date_widget = widgets.Text(value='YYYY-MM-DD', description='Startdatum:')
end_date_widget = widgets.Text(value='YYYY-MM-DD', description='Einddatum:')
button_generate_income_widgets = widgets.Button(description="Je jaarinkomens")
button_generate_income_widgets.on_click(generate_yearly_income_widgets)

# Knop voor berekening
button_calculate = widgets.Button(description="Bereken")
button_calculate.on_click(on_calculate_button_click)

# Weergeven van widgets
display(start_date_widget, end_date_widget, button_generate_income_widgets)
